In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import torch
import matplotlib.pyplot as plt
from bert_embedding import BertEmbedding


In [2]:
#data_orig = pd.read_json('../data/annotated_data_full.json')
data_orig = pd.read_json('../data/annotated_posts.json')
data_orig = data_orig

In [3]:
data_orig

,type,body,id,is_first_post,depth,author,reply_to
0,announcement,4/7/13 \n\n7/27/12 \n\nhttp://www.imdb.com/t...,t3_1bx6qw,1,0,DTX120,None
1,elaboration,I've wanted to watch this for a long time. I w...,t1_c9b2nyd,0,1,mcgrewf10,t3_1bx6qw
2,elaboration,You strike me as the type who would appreciate...,t1_c9b30i1,0,2,DTX120,t1_c9b2nyd
3,elaboration,"Yeah, I've always heard that Altman was famous...",t1_c9b6sj0,0,3,mcgrewf10,t1_c9b30i1
4,announcement,"Alright guys, little background about myself. ...",t3_omv7p,1,0,Keatonus,None
...,...,...,...,...,...,...,...
101700,question,Did anyone else spot the Floatzen 2 DVD?\nI kn...,t3_48toyo,1,0,Pradfanne,None
101701,answer,I didn't notice it until it was too late. What...,t1_d0mhap4,0,1,Portgas,t3_48toyo
101702,answer,"I don't know, it was to quick.\nI just liked t...",t1_d0n2wdx,0,2,Pradfanne,t1_d0mhap4
101703,answer,The Frozen parody was on the right edge of the...,t1_d0msypp,0,2,SaucyServine,t1_d0mhap4


In [4]:
#calculate number of posts in thread and whetehr commenter is  OP

post_count = []
is_OP = []
word_count = []
sent_count = []
parent_word_count = []
parent_sent_count = []

counter = 1
for ix in range(0,len(data_orig)):
    
    if data_orig.loc[ix].is_first_post == 1:
        post_count += counter*[counter]
        counter = 1
        
        #checking wether the commenter is also the Original Poster
        OP = data_orig.loc[ix].author
        is_OP.append(1)
        
        parent_sent_count.append(0)
        parent_word_count.append(0)
        
    else:
        counter += 1
        
        #checking wether the commenter is also the Original Poster
        if data_orig.loc[ix].author == OP:
            is_OP.append(1)
        else:
            is_OP.append(0)
        try:
            parent_sent_count.append(sent_count[data_orig[data_orig.id == data_orig.loc[ix].reply_to].index[0]])
            parent_word_count.append(word_count[data_orig[data_orig.id == data_orig.loc[ix].reply_to].index[0]])
        except:
            parent_sent_count.append(0)
            parent_word_count.append(0)
            #print('PARENT NOT FOUND')
   
     # Idk how to vectorize this, so i just do it here but slowly
    words = word_tokenize(data_orig.loc[ix].body)
    sents = sent_tokenize(data_orig.loc[ix].body)
    word_count.append(len(words))
    sent_count.append(len(sents))
    

post_count += counter*[counter]
del post_count[0]

data_orig['nr_of_posts'] = post_count
data_orig['is_OP'] = is_OP
data_orig['sent_count'] = sent_count
data_orig['word_count'] = word_count
data_orig['parent_sent_count'] = parent_sent_count
data_orig['parent_word_count'] = parent_sent_count

In [5]:
cols = data_orig.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols

['parent_word_count',
 'type',
 'body',
 'id',
 'is_first_post',
 'depth',
 'author',
 'reply_to',
 'nr_of_posts',
 'is_OP',
 'sent_count',
 'word_count',
 'parent_sent_count']

In [6]:
#these are manually selected obvious bot accounts with bot in their name
bot_accounts = ['autowikibot', 'flair_your_post_bot', 'gracefulcharitybot', 'domoarigatobtfcboto', 'reddtipbot',
               'Mariners_bot', 'rightsbot', 'ttumblrbots', 'image_linker_bot', 'conspirobot', 'dogetipbot', 'juicedb_bot',
               'telltalebot', 'hearthscan-bot', 'gifv-bot']

In [7]:
df = data_orig[cols]
for bot in range(0, len(bot_accounts)):
    df = df[df['author'] != bot_accounts[bot]]
df = df[df.body != '']
df = df[df.body != '[deleted]']


punct = '\t\n\r'

processed_posts = []


for post in df.body:
    post = post.translate(str.maketrans('', '', punct))
    processed_posts.append(post)

df.drop(columns=['body'])
df['body'] = processed_posts

In [8]:
df['normalized_depth'] = (df.depth)/(df.nr_of_posts)
df['character_count'] = df.body.str.len()


In [9]:
df['structure'] = np.column_stack((df.depth,df.character_count, df.word_count,df.normalized_depth,  df.sent_count, df.is_OP, df.parent_word_count, df.parent_sent_count)).tolist()


#df['structure'] = np.column_stack((df.depth, df.normalized_depth, df.character_count, df.word_count,  df.sent_count)).tolist()


In [10]:

#df.loc[(df['type'] != 'question') & (df['type'] != 'answer'), 'type'] = 'other'
df.type.describe()

count      98868
unique        10
top       answer
freq       40153
Name: type, dtype: object

In [11]:
test = df[0:4]
test.body[3]

"Yeah, I've always heard that Altman was famous for his ensemble casts. But I, too, have never seen an Altman."

In [15]:
df.to_json('../data/annotated_posts_clean')

In [12]:
train, test = train_test_split(df, test_size=0.2)

In [13]:
train.to_json('../data/train.json', orient='records', lines = True)
test.to_json('../data/test.json', orient='records', lines = True)

In [ ]:
#calculate post depth 

'''
data_orig['reply_to'] = data_orig.reply_to.fillna("root")
deepts = []
exceptions = 0

for ix in range(0, len(data_orig)):
    depth = 0
    
    reply_to = data_orig.loc[ix].reply_to
    ID = data_orig.loc[ix].id
    
    try:
        while(reply_to != 'root'):
            rt_ix = data_orig.index[data_orig['id'] == reply_to]
            rt_ix = rt_ix[0]
            reply_to = data_orig.loc[rt_ix].reply_to
            depth += 1

        deepts.append([depth])
        
    except:
        exceptions += 1
        deepts.append([0])

print('exceptions', exceptions)
print(deepts)

data_orig['depth'] = deepts
'''